# Reasoning and tool use

I'm going to use some more advanced chain of thought loops to give the scratchpad more structure. I will methods which combine reasoning and actions to let an agent engage with a world.


### Agents
- simple agent loop
- rudimentary tools
- custom agent prompt
- advanced tools
- custom character 

In [48]:
# Setup environment
import textworld
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
from time import sleep

load_dotenv()
llm = OpenAI(model_name="text-davinci-003", temperature=0.0, max_tokens=50)


In [37]:

# Let the environment know what information we want as part of the game state.
infos = textworld.EnvInfos(
    feedback=True,    # Response from the game after typing a text command.
    description=True, # Text describing the room the player is currently in.
    inventory=True,    # Text describing the player's inventory.
    max_score=True,   # Maximum score obtainable in the game.
    score=True,       # Score obtained so far.
)
env = textworld.start('./zork1.z5', infos=infos)
game_state = env.reset()


Have to think about this differently now. Previously it was the game maintaining all the state, and the environment was a stateless LLMchain. Now I want the agent to think about what to do, and ideally maintain one cohesive conversation with the environment.

Testing first with the default ReAct agent, and then I'll try to make a custom agent.

Need a single tool just to start playing the game, of course:

In [83]:
# Simple tool to play the game
def send_command(command):
    """
    Send a commmand to the game and receive feedback."""
    game_state, score, done = env.step(command)
    return game_state.feedback
play = Tool("Play", send_command, send_command.__doc__)


In [112]:
tools = [play]
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True,
    max_iterations=10,
    )

game_state = env.reset()
print(game_state.description)
agent.run(game_state.description)

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.




> Entering new AgentExecutor chain...
 I should investigate the mailbox
Action: Play
Action Input: open mailbox
Observation: Opening the small mailbox reveals a leaflet.


Thought: I should read the leaflet
Action: Play
Action Input: read leaflet
Observation: (Taken)
"WELCOME TO ZORK!

ZORK is a game of adventure, danger, and low cunning. In it you will explore some of the most amazing territory ever seen by mortals. No computer should be without one!"



Thought: I should find out more about Zork
Action: Play
Action Input: what is zork
Observation: You can't see any zork here!


Thought: I should look for more information about Zork
Action: Play
Action Input: look for zork
Observation: You can't see any zork here!


Thought: I should search online for more information about Zork
Action: Play
Action Input: search online for zork
Observation: I don't know 

'Agent stopped due to max iterations.'

It looks like there's a semi-documented TextWorld agent, so let's try that instead.

In [129]:
from langchain.agents import ReActTextWorldAgent, AgentExecutor

rtwa_prompt = ReActTextWorldAgent.create_prompt(tools)
print(rtwa_prompt.template)



Setup: You are now playing a fast paced round of TextWorld! Here is your task for
today. First of all, you could, like, try to travel east. After that, take the
binder from the locker. With the binder, place the binder on the mantelpiece.
Alright, thanks!

-= Vault =-
You've just walked into a vault. You begin to take stock of what's here.

An open safe is here. What a letdown! The safe is empty! You make out a shelf.
But the thing hasn't got anything on it. What, you think everything in TextWorld
should have stuff on it?

You don't like doors? Why not try going east, that entranceway is unguarded.

Thought 1: I need to travel east
Action 1: Play[go east]
Observation 1: -= Office =-
You arrive in an office. An ordinary one.

You can make out a locker. The locker contains a binder. You see a case. The
case is empty, what a horrible day! You lean against the wall, inadvertently
pressing a secret button. The wall opens up to reveal a mantelpiece. You wonder
idly who left that here. The 

In [130]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=rtwa_prompt, verbose=True)

agent = ReActTextWorldAgent(llm_chain=llm_chain, tools=tools)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [131]:
game_state = env.reset()
prefix = f""" You are now playing a fast paced round of TextWorld! Here is your task for
today. Explore the dungeon, find the treasure and return a hero!
{game_state.description}"""

print(prefix)
agent_executor.run(prefix)
print("Played {} steps, scoring {} points.".format(game_state.moves, game_state.score))

 You are now playing a fast paced round of TextWorld! Here is your task for
today. Explore the dungeon, find the treasure and return a hero!
West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:


Setup: You are now playing a fast paced round of TextWorld! Here is your task for
today. First of all, you could, like, try to travel east. After that, take the
binder from the locker. With the binder, place the binder on the mantelpiece.
Alright, thanks!

-= Vault =-
You've just walked into a vault. You begin to take stock of what's here.

An open safe is here. What a letdown! The safe is empty! You make out a shelf.
But the thing hasn't got anything on it. What, you think everything in TextWorld
should have stuff on it?

You don't like doors? Why not try going east, that entranceway is unguarded.

Thought 1: I need t

RateLimitError: You exceeded your current quota, please check your plan and billing details.